In [ ]:
https://www.podrygka.ru/
    
https://www.podrygka.ru/robots.txt
    
https://www.m.podrygka.ru/m.sitemap_contacts.xml
    
https://www.m.podrygka.ru/sitemap/
    
https://www.podrygka.ru/sitemap.xml
    https://www.podrygka.ru/sitemap_products.xml - продукты
    https://www.podrygka.ru/sitemap_brands.xml - бренды
        
Апишка с сайта:
https://www.m.podrygka.ru/api/v2/mobile/menu
https://www.m.podrygka.ru/api/v2/user/status
    
в скриптах много отсылок к апи, в исходнике супа
rrApi.addToBasket = rrApi.order
    
# эксплойты тут точно есть, потом можно оставить для оптимизации
# пока парсинг медленный из-за его прямого дизайна

In [30]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                #every = int(size / 200)     # every 0.5%
                every = 1
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )


In [35]:
import requests
import json
import pandas as pd 
from bs4 import BeautifulSoup
import time
import re
from datetime import datetime
from datetime import date

based_url = ' https://www.podrygka.ru'

# список всех брендов

headers = {
    'Content-Type': 'application/json', 
}

response = requests.get('https://www.podrygka.ru/sitemap_brands.xml')

parsed_xml_list = response.text.replace('</loc>', '<loc>').split('<loc>')
filtered_parsed_list = []

for i in parsed_xml_list:
    if 'podrygka.ru' in i:
        filtered_parsed_list.append(i)
        
len(filtered_parsed_list)

643

In [36]:
iter_brand_urls = filtered_parsed_list[0]
response = requests.get(iter_brand_urls)
print(iter_brand_urls)

soup = BeautifulSoup(response.text, 'lxml')

soup.find_all(class_="pagination__button pagination__button--right")[0].get('href')

https://www.podrygka.ru/catalog/brand-is-loreal/


'/catalog/?brand=loreal&PAGEN_1=2'

In [50]:
parsed_podrygka_data = []
prev_number = 0
xxx = ['', '']

for iter_brand_urls in log_progress(filtered_parsed_list):
    pars_checker = 2
    while True:
        if pars_checker == 2:
            #парсинг первой страницы
            # уровень итерации по брендам
            response = requests.get(iter_brand_urls)
            print('first level:', iter_brand_urls)
            #logger.info("first level: {%s}", iter_brand_urls)

            soup = BeautifulSoup(response.text, 'lxml')
            pars_checker = 1
            
        if pars_checker == 0:
            #парсинг некст страницы
            # уровень итерации по брендам
            response = requests.get(based_url + url_part_for_next)
            print('next level:', based_url + url_part_for_next)
            #logger.info("next level: {%s}", based_url + url_part_for_next)
            
            soup = BeautifulSoup(response.text, 'lxml')
            pars_checker = 1

        if pars_checker == 1:
            # обработка страницы бренда
            # только если произошла загрузка новой страницы
            for iter_product_id in range(0, len(soup.find_all(class_="item-product-card"))):
                temp_product_dict = {}

                # айди
                product_id = soup.find_all(class_="item-product-card")[iter_product_id].find_all(class_="products-list-item vertically_sides_in_box")[0].get('data-article')

                # название
                product_name = soup.find_all(class_="item-product-card")[iter_product_id].find_all(class_="products-list-item__title")[0].text

                # бренд
                product_brand = soup.find_all(class_="item-product-card")[iter_product_id].find_all(class_="brand")[0].text

                # ссылка
                product_url = based_url+ soup.find_all(class_="item-product-card")[iter_product_id].find_all(class_="products-list-item__title")[0].get('href')

                # цена продукта
                temp_find_price = soup.find_all(class_="item-product-card")[iter_product_id].find_all(class_="price")[0].text
                product_price = float(temp_find_price)
                
                try:
                    # комментарий по продукту (категория)
                    product_comment = soup.find_all(class_="item-product-card")[iter_product_id].find_all(class_="category")[0].text
                except:
                    product_comment = ''


                temp_product_dict['product_id'] = product_id
                temp_product_dict['product_name'] = product_name
                temp_product_dict['product_brand'] = product_brand
                temp_product_dict['product_url'] = product_url
                temp_product_dict['product_price'] = product_price
                temp_product_dict['product_comment'] = product_comment

                parsed_podrygka_data.append(temp_product_dict)
            print('parsed_podrygka_data len:', len(parsed_podrygka_data))
            #logger.info("parsed_podrygka_data len: {%s}", len(parsed_podrygka_data))

        try:
            # проверка на наличие некст страницы
            url_part_for_next = soup.find_all(class_="pagination__button pagination__button--right")[0].get('href')
            #print('url_part_for_next:', url_part_for_next)
            if url_part_for_next == None:
                xxx[5] # вызываем индекс аут оф рейндж
            pars_checker = 0
            # попробуем без таймера между страничками
            #time.sleep(1)
        except Exception as e:
            if 'list index out of range' not in str(e):
                # предотвращение загрузки следующей страницы, все данные бренда получены, последняя страница достигнута
                stop
            got_data_number = len(parsed_podrygka_data) - prev_number
            prev_number = len(parsed_podrygka_data)
            print('got data:', got_data_number)
            #logger.info("got data: {%s}", got_data_number)
            pars_checker = 2
            time.sleep(1)
            #print(e)
            break

first level: https://www.podrygka.ru/catalog/brand-is-loreal/
parsed_podrygka_data len: 20
next level:  https://www.podrygka.ru/catalog/?brand=loreal&PAGEN_1=2
parsed_podrygka_data len: 40
next level:  https://www.podrygka.ru/catalog/?brand=loreal&PAGEN_1=3
parsed_podrygka_data len: 60
next level:  https://www.podrygka.ru/catalog/?brand=loreal&PAGEN_1=4
parsed_podrygka_data len: 80
next level:  https://www.podrygka.ru/catalog/?brand=loreal&PAGEN_1=5
parsed_podrygka_data len: 100
next level:  https://www.podrygka.ru/catalog/?brand=loreal&PAGEN_1=6
parsed_podrygka_data len: 120
next level:  https://www.podrygka.ru/catalog/?brand=loreal&PAGEN_1=7
parsed_podrygka_data len: 140
next level:  https://www.podrygka.ru/catalog/?brand=loreal&PAGEN_1=8
parsed_podrygka_data len: 160
next level:  https://www.podrygka.ru/catalog/?brand=loreal&PAGEN_1=9
parsed_podrygka_data len: 180
next level:  https://www.podrygka.ru/catalog/?brand=loreal&PAGEN_1=10
parsed_podrygka_data len: 200
next level:  https:/

parsed_podrygka_data len: 1447
next level:  https://www.podrygka.ru/catalog/?brand=miss_pinky&PAGEN_1=19
parsed_podrygka_data len: 1467
next level:  https://www.podrygka.ru/catalog/?brand=miss_pinky&PAGEN_1=20
parsed_podrygka_data len: 1487
next level:  https://www.podrygka.ru/catalog/?brand=miss_pinky&PAGEN_1=21
parsed_podrygka_data len: 1507
next level:  https://www.podrygka.ru/catalog/?brand=miss_pinky&PAGEN_1=22
parsed_podrygka_data len: 1527
next level:  https://www.podrygka.ru/catalog/?brand=miss_pinky&PAGEN_1=23
parsed_podrygka_data len: 1547
next level:  https://www.podrygka.ru/catalog/?brand=miss_pinky&PAGEN_1=24
parsed_podrygka_data len: 1567
next level:  https://www.podrygka.ru/catalog/?brand=miss_pinky&PAGEN_1=25
parsed_podrygka_data len: 1587
next level:  https://www.podrygka.ru/catalog/?brand=miss_pinky&PAGEN_1=26
parsed_podrygka_data len: 1607
next level:  https://www.podrygka.ru/catalog/?brand=miss_pinky&PAGEN_1=27
parsed_podrygka_data len: 1627
next level:  https://www

parsed_podrygka_data len: 2755
got data: 4
first level: https://www.podrygka.ru/catalog/brand-is-got2b/
parsed_podrygka_data len: 2773
got data: 18
first level: https://www.podrygka.ru/catalog/brand-is-eco_therapy/
parsed_podrygka_data len: 2781
got data: 8
first level: https://www.podrygka.ru/catalog/brand-is-o.b./
parsed_podrygka_data len: 2787
got data: 6
first level: https://www.podrygka.ru/catalog/brand-is-ace/
parsed_podrygka_data len: 2789
got data: 2
first level: https://www.podrygka.ru/catalog/brand-is-lumene/
parsed_podrygka_data len: 2803
got data: 14
first level: https://www.podrygka.ru/catalog/brand-is-old_spice/
parsed_podrygka_data len: 2812
got data: 9
first level: https://www.podrygka.ru/catalog/brand-is-hello_nature/
parsed_podrygka_data len: 2812
got data: 0
first level: https://www.podrygka.ru/catalog/brand-is-gliss_kur/
parsed_podrygka_data len: 2832
next level:  https://www.podrygka.ru/catalog/?brand=gliss_kur&PAGEN_1=2
parsed_podrygka_data len: 2852
next level:  

first level: https://www.podrygka.ru/catalog/brand-is-perfect_mousse/
parsed_podrygka_data len: 3836
next level:  https://www.podrygka.ru/catalog/?brand=perfect_mousse&PAGEN_1=2
parsed_podrygka_data len: 3837
got data: 21
first level: https://www.podrygka.ru/catalog/brand-is-art_parfum/
parsed_podrygka_data len: 3841
got data: 4
first level: https://www.podrygka.ru/catalog/brand-is-california_mango/
parsed_podrygka_data len: 3845
got data: 4
first level: https://www.podrygka.ru/catalog/brand-is-batiste/
parsed_podrygka_data len: 3855
got data: 10
first level: https://www.podrygka.ru/catalog/brand-is-freemay/
parsed_podrygka_data len: 3859
got data: 4
first level: https://www.podrygka.ru/catalog/brand-is-planeta_organica/
parsed_podrygka_data len: 3879
next level:  https://www.podrygka.ru/catalog/?brand=planeta_organica&PAGEN_1=2
parsed_podrygka_data len: 3899
next level:  https://www.podrygka.ru/catalog/?brand=planeta_organica&PAGEN_1=3
parsed_podrygka_data len: 3901
got data: 42
first

parsed_podrygka_data len: 4521
got data: 45
first level: https://www.podrygka.ru/catalog/brand-is-splat/
parsed_podrygka_data len: 4541
next level:  https://www.podrygka.ru/catalog/?brand=splat&PAGEN_1=2
parsed_podrygka_data len: 4544
got data: 23
first level: https://www.podrygka.ru/catalog/brand-is-orjena/
parsed_podrygka_data len: 4547
got data: 3
first level: https://www.podrygka.ru/catalog/brand-is-malenkaya_feya/
parsed_podrygka_data len: 4549
got data: 2
first level: https://www.podrygka.ru/catalog/brand-is-olea/
parsed_podrygka_data len: 4555
got data: 6
first level: https://www.podrygka.ru/catalog/brand-is-wet_n_wild/
parsed_podrygka_data len: 4555
got data: 0
first level: https://www.podrygka.ru/catalog/brand-is-fitocolor/
parsed_podrygka_data len: 4567
got data: 12
first level: https://www.podrygka.ru/catalog/brand-is-sun_look/
parsed_podrygka_data len: 4587
next level:  https://www.podrygka.ru/catalog/?brand=sun_look&PAGEN_1=2
parsed_podrygka_data len: 4607
next level:  htt

...

first level: https://www.podrygka.ru/catalog/brand-is-obb/
parsed_podrygka_data len: 11903
got data: 7
first level: https://www.podrygka.ru/catalog/brand-is-the_lab/
parsed_podrygka_data len: 11908
got data: 5
first level: https://www.podrygka.ru/catalog/brand-is-kotex/
parsed_podrygka_data len: 11928
next level:  https://www.podrygka.ru/catalog/?brand=kotex&PAGEN_1=2
parsed_podrygka_data len: 11938
got data: 30
first level: https://www.podrygka.ru/catalog/brand-is-lacalut/
parsed_podrygka_data len: 11944
got data: 6
first level: https://www.podrygka.ru/catalog/brand-is-mentos/
parsed_podrygka_data len: 11945
got data: 1
first level: https://www.podrygka.ru/catalog/brand-is-rude/
parsed_podrygka_data len: 11945
got data: 0
first level: https://www.podrygka.ru/catalog/brand-is-kapous/
parsed_podrygka_data len: 11946
got data: 1
first level: https://www.podrygka.ru/catalog/brand-is-sanfor/
parsed_podrygka_data len: 11949
got data: 3
first level: https://www.podrygka.ru/catalog/brand-is-p

parsed_podrygka_data len: 13332
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=70
parsed_podrygka_data len: 13352
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=71
parsed_podrygka_data len: 13372
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=72
parsed_podrygka_data len: 13392
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=73
parsed_podrygka_data len: 13412
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=74
parsed_podrygka_data len: 13432
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=75
parsed_podrygka_data len: 13452
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=76
parsed_podrygka_data len: 13472
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=77
parsed_podrygka_data len: 13492
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=78
parsed_podrygka_data len: 13512
next level:  https://ww

parsed_podrygka_data len: 14892
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=148
parsed_podrygka_data len: 14912
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=149
parsed_podrygka_data len: 14932
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=150
parsed_podrygka_data len: 14952
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=151
parsed_podrygka_data len: 14972
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=152
parsed_podrygka_data len: 14992
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=153
parsed_podrygka_data len: 15012
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=154
parsed_podrygka_data len: 15032
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=155
parsed_podrygka_data len: 15052
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=156
parsed_podrygka_data len: 15072
next level:  h

next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=225
parsed_podrygka_data len: 16452
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=226
parsed_podrygka_data len: 16472
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=227
parsed_podrygka_data len: 16492
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=228
parsed_podrygka_data len: 16512
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=229
parsed_podrygka_data len: 16532
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=230
parsed_podrygka_data len: 16552
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=231
parsed_podrygka_data len: 16572
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=232
parsed_podrygka_data len: 16592
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=233
parsed_podrygka_data len: 16612
next level:  https://www.podrygka.ru/catalog/?

parsed_podrygka_data len: 17992
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=303
parsed_podrygka_data len: 18012
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=304
parsed_podrygka_data len: 18032
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=305
parsed_podrygka_data len: 18052
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=306
parsed_podrygka_data len: 18072
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=307
parsed_podrygka_data len: 18092
next level:  https://www.podrygka.ru/catalog/?brand=lady_pink&PAGEN_1=308
parsed_podrygka_data len: 18110
got data: 6158
first level: https://www.podrygka.ru/catalog/brand-is-mini_dolly/
parsed_podrygka_data len: 18117
got data: 7
first level: https://www.podrygka.ru/catalog/brand-is-expel/
parsed_podrygka_data len: 18119
got data: 2
first level: https://www.podrygka.ru/catalog/brand-is-bia_effect/
parsed_podrygka_data len: 18124
got 

In [51]:
len(parsed_podrygka_data)

18777

In [52]:
# 19:59
now = datetime.now()
print('finish:', now)

finish: 2022-11-22 22:25:57.152637


In [54]:
with open('data_podrygka.json', 'w', encoding='utf-8') as f:
    json.dump(parsed_podrygka_data, f, ensure_ascii=False, indent=4)

'133957'

In [ ]:
# айди
soup.find_all(class_="item-product-card")[0].find_all(class_="products-list-item vertically_sides_in_box")[0].get('data-article')

# название
soup.find_all(class_="item-product-card")[0].find_all(class_="products-list-item__title")[0].text

# бренд
soup.find_all(class_="item-product-card")[0].find_all(class_="brand")[0].text

# ссылка
soup.find_all(class_="item-product-card")[0].find_all(class_="products-list-item__title")[0].get('href')

# цена продукта
soup.find_all(class_="item-product-card")[0].find_all(class_="price")[0].text

# категория
soup.find_all(class_="item-product-card")[0].find_all(class_="category")[0].text

In [40]:
soup.find_all(class_="products-list-item vertically_sides_in_box")[0].get('data-article')

'133957'

In [39]:
soup.text

'\n\n\nL’Oreal Paris – купить косметику бренда по низкой цене в «Подружке»\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n(function(w, d, n) {var cl = "bx-core";var ht = d.documentElement;var htc = ht ? ht.className : undefined;if (htc === undefined || htc.indexOf(cl) !== -1){return;}var ua = n.userAgent;if (/(iPad;)|(iPhone;)/i.test(ua)){cl += " bx-ios";}else if (/Android/i.test(ua)){cl += " bx-android";}cl += (/(ipad|iphone|android|mobile|touch)/i.test(ua) ? " bx-touch" : " bx-no-touch");cl += w.devicePixelRatio && w.devicePixelRatio >= 2? " bx-retina": " bx-no-retina";var ieVersion = -1;if (/AppleWebKit/.test(ua)){cl += " bx-chrome";}else if ((ieVersion = getIeVersion()) > 0){cl += " bx-ie bx-ie" + ieVersion;if (ieVersion > 7 && ieVersion < 10 && !isDoctype()){cl += " bx-quirks";}}else if (/Opera/.test(ua)){cl += " bx-opera";}else if (/Gecko/.test(ua)){cl += " bx-firefox";}if (/Macintosh/i.test(ua)){cl += " bx-mac";}ht.className = htc ? htc + " " + cl : cl;function isDoctype(){if (d.compa